# Ingesting Data

In [3]:
import numpy as np
import pandas as pd

brands_df = pd.read_json(r"JSON Data\brands.json.gz", compression='gzip', lines=True)
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)
users_df = pd.read_json(r"JSON Data\users.json.gz", compression='gzip', lines=True)

***
## Receipts

In [68]:
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)
# Step 1: Extract '$oid' from the _id column
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)

# Step 2: Extract '$date' from various date columns
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for column in date_columns:
    receipts_df[column] = receipts_df[column].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else None)

# Display the cleaned DataFrame
receipts_df


,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,NaN,1614594148000,NaN,25.0,1.597622e+12,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,1614613361873,1614613361873,NaN,1614613361873,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,1614607657664,1614607657664,NaN,1614607657664,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,NaN,1614604049000,NaN,25.0,1.597622e+12,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [77]:
# Quick Null check 
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)
receipts_df.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

It looks like all of the `date` fields are in Unixtime with milliseconds. Since Unixtime in seconds is a more standard record keeping time, I think I would upload this data to the relational database in seconds by using integer division on the `date` fields

For example, to convert the `createDate` field to seconds, I would do `createDate = createDate//1000`

Inside of the relational database, assume that all Unixtimes are truncated to the second instead of the provided millisecond

***
### Isolating dataframe to the rewardsReceiptItemList and it's matching receipt _id

In [96]:
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)
# Step 1: Extract '$oid' from the _id column
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)

# Isolate the DataFrame to just _id and rewardsReceiptItemList
isolated_df = receipts_df[['_id', 'rewardsReceiptItemList']]

# Explode the 'rewardsReceiptItemList' to expand the array into rows
exploded_df = isolated_df.explode('rewardsReceiptItemList')

# Display the expanded DataFrame
exploded_df

,_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO..."
2,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '..."
3,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F..."
...,...,...
1115,603d0b710a720fde1000042a,NaN
1116,603cf5290a720fde10000413,NaN
1117,603ce7100a7217c72c000405,"{'barcode': 'B076FJ92M4', 'description': 'muel..."
1117,603ce7100a7217c72c000405,"{'barcode': 'B07BRRLSVC', 'description': 'thin..."


In [114]:
# Initialize an empty set to collect unique keys
distinct_keys = set()

# Iterate through the column to update the set with keys from each dictionary
for item in exploded_df['rewardsReceiptItemList']:
    if isinstance(item, dict):  # Ensure that the item is a dictionary
        distinct_keys.update(item.keys())

# Print the distinct keys found
distinct_keys


{'barcode',
 'brandCode',
 'competitiveProduct',
 'competitorRewardsGroup',
 'deleted',
 'description',
 'discountedItemPrice',
 'finalPrice',
 'itemNumber',
 'itemPrice',
 'metabriteCampaignId',
 'needsFetchReview',
 'needsFetchReviewReason',
 'originalFinalPrice',
 'originalMetaBriteBarcode',
 'originalMetaBriteDescription',
 'originalMetaBriteItemPrice',
 'originalMetaBriteQuantityPurchased',
 'originalReceiptItemText',
 'partnerItemId',
 'pointsEarned',
 'pointsNotAwardedReason',
 'pointsPayerId',
 'preventTargetGapPoints',
 'priceAfterCoupon',
 'quantityPurchased',
 'rewardsGroup',
 'rewardsProductPartnerId',
 'targetPrice',
 'userFlaggedBarcode',
 'userFlaggedDescription',
 'userFlaggedNewItem',
 'userFlaggedPrice',
 'userFlaggedQuantity'}

^^ That is a lot of unique keys! I think it would be helpful to have a new table that shows a 1:1 relationship of the barcodes to receipt_id so it's easier to reference all of the items inside of each receipt, without worrying about duplicating the other receipt-centric data

In [98]:
exploded_df['barcode'] = exploded_df['rewardsReceiptItemList'].apply(lambda x: x.get('barcode') if isinstance(x, dict) and 'barcode' in x else None)
exploded_df['description'] = exploded_df['rewardsReceiptItemList'].apply(lambda x: x.get('description') if isinstance(x, dict) and 'description' in x else None)


barcode_receipt_df = exploded_df[['_id', 'barcode','description']]
barcode_receipt_df

,_id,barcode,description
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND
1,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...
2,5ff1e1f10a720f052300057a,None,None
3,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND
...,...,...,...
1115,603d0b710a720fde1000042a,None,None
1116,603cf5290a720fde10000413,None,None
1117,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...
1117,603ce7100a7217c72c000405,B07BRRLSVC,thindust summer face mask - sun protection nec...


There is some interesting barcodes and descriptions here, so I am going to check if there is common "missing" or "error" type barcodes

In [107]:
# Step 1: Count occurrences of each unique description and sort them
description_counts = barcode_receipt_df['description'].value_counts(dropna=False).reset_index(name='count')

# # Rename the columns for clarity
# description_counts.columns = ['description', 'count']

# Step 2: Sort the DataFrame by 'count' in descending order to see the most frequent descriptions first
description_counts_sorted = description_counts.sort_values(by='count', ascending=False)

description_counts_sorted


,description,count
0,None,821
1,ITEM NOT FOUND,173
2,KLARBRUNN 12PK 12 FL OZ,120
3,HUGGIES SIMPLY CLEAN PREMOISTENED WIPE FRAGRAN...,92
4,MILLER LITE 24 PACK 12OZ CAN,90
...,...,...
1138,EMI1 S SAUSAGE MUSHROOM PIZZA,1
1139,LEGO41165,1
1140,BC HRSHYCHOC FR,1
1141,GMCEREALCRN,1


In [106]:
# Step 1: Use value_counts() on both 'barcode' and 'description' to count occurrences
description_barcode_counts = barcode_receipt_df[['barcode', 'description']].value_counts(dropna=False).reset_index(name='count')

# Step 2: Sort the DataFrame by 'count' in descending order to see the most frequent combinations first
description_barcode_counts_sorted = description_barcode_counts.sort_values(by='count', ascending=False).reset_index(drop=True)

description_barcode_counts_sorted

,barcode,description,count
0,NaN,NaN,590
1,4011,ITEM NOT FOUND,160
2,NaN,KLARBRUNN 12PK 12 FL OZ,120
3,036000320893,HUGGIES SIMPLY CLEAN PREMOISTENED WIPE FRAGRAN...,92
4,034100573065,MILLER LITE 24 PACK 12OZ CAN,90
...,...,...,...
1955,NaN,SIMPLYORANGEJCE,1
1956,NaN,SKITTLES VNLD,1
1957,019582000117,Bull's Eye Brewer's Best Barbecue Sauce with G...,1
1958,NaN,SIGICE CRV MOOSF,1


***
### Testing quantity purchased to see if this aligns with the purchasedItemCount column


In [116]:
# Relying on the section above to have already created the dataframe `exploded_df`

# Step 1: Extracting quantity purchased to see how many items are purchased per barcode
exploded_df['quantityPurchased'] = exploded_df['rewardsReceiptItemList'].apply(lambda x: x.get('quantityPurchased') if isinstance(x, dict) else None)

# Step 2: Create a new DataFrame with the specified columns
quantity_df = exploded_df[['_id', 'barcode', 'quantityPurchased']]
quantity_df


,_id,barcode,quantityPurchased
0,5ff1e1eb0a720f0523000575,4011,5.0
1,5ff1e1bb0a720f052300056b,4011,1.0
1,5ff1e1bb0a720f052300056b,028400642255,1.0
2,5ff1e1f10a720f052300057a,None,NaN
3,5ff1e1ee0a7214ada100056f,4011,4.0
...,...,...,...
1115,603d0b710a720fde1000042a,None,NaN
1116,603cf5290a720fde10000413,None,NaN
1117,603ce7100a7217c72c000405,B076FJ92M4,1.0
1117,603ce7100a7217c72c000405,B07BRRLSVC,1.0


In [117]:
# Aggregating the dataframe up to the _id level to get the sum of quantityPurchased per receipt `_id`

# Step 1: Group by '_id' and sum 'quantityPurchased'
quantity_sum_per_id = quantity_df.groupby('_id')['quantityPurchased'].sum().reset_index()

# Rename the columns for clarity
quantity_sum_per_id.columns = ['_id', 'total_quantity_purchased']

quantity_sum_per_id

,_id,total_quantity_purchased
0,5f9c74f70a7214ad07000037,1.0
1,5f9c74f90a7214ad07000038,6.0
2,5fa5ad370a720f05ef000089,11.0
3,5fa5b0ca0a720f05ef0000bf,6.0
4,5fa8d5730a7214adc30001c3,1.0
...,...,...
1114,603d30e60a7217c72c00043f,2.0
1115,603d40250a720fde10000459,2.0
1116,603d59e70a7217c72c00045f,2.0
1117,603d5d6c0a7217c72c000463,2.0


In [124]:
# Merging the original receipts_df to compare that the quantities match properly

# Refreshing the dataframe
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)
# Extracting _id to make it joinable
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)

# Joining the dataframes together
merged_df = pd.merge(receipts_df[['_id', 'purchasedItemCount']],
                     quantity_sum_per_id,
                     on='_id',
                     how='inner')

# Getting the difference between 
merged_df['quantity_difference'] = merged_df['total_quantity_purchased'] - merged_df['purchasedItemCount']
merged_df


,_id,purchasedItemCount,total_quantity_purchased,quantity_difference
0,5ff1e1eb0a720f0523000575,5.0,5.0,0.0
1,5ff1e1bb0a720f052300056b,2.0,2.0,0.0
2,5ff1e1f10a720f052300057a,1.0,0.0,-1.0
3,5ff1e1ee0a7214ada100056f,4.0,4.0,0.0
4,5ff1e1d20a7214ada1000561,2.0,4.0,2.0
...,...,...,...,...
1114,603cc0630a720fde100003e6,2.0,2.0,0.0
1115,603d0b710a720fde1000042a,NaN,0.0,NaN
1116,603cf5290a720fde10000413,NaN,0.0,NaN
1117,603ce7100a7217c72c000405,2.0,2.0,0.0


In [125]:
# Aggregate by 'quantity_difference' to count occurrences
difference_agg_df = merged_df.groupby('quantity_difference').size().reset_index(name='count')

# Sorting the resulting DataFrame by 'count' in descending order
difference_agg_df = difference_agg_df.sort_values(by='count', ascending=False).reset_index(drop=True)

# Display the resulting DataFrame
difference_agg_df

,quantity_difference,count
0,0.0,591
1,-1.0,28
2,2.0,8
3,-10.0,4
4,4.0,2
5,1.0,1
6,3.0,1


This looks like the `SUM(quantityPurchased)` is generally supposed to align with the purchasedItemCount, so for the SQL analysis, I am going to keep that assumption for the table I am adding in my relational database `receipt_barcode` which will hold the `receipt_id`, the `barcode`, and the `quantityPurchased` for that barcode, so we can provide the proper volume of items being purchased per brand.

It could also be helpful to include `itemPrice` and `discountedItemPrice` if we wanted analysis for something else, so I would also include them - this would be a nice sample of the dataframe

In [129]:
receipt_barcode_df = exploded_df
receipt_barcode_df['itemPrice'] = receipt_barcode_df['rewardsReceiptItemList'].apply(lambda x: x.get('itemPrice') if isinstance(x, dict) and 'itemPrice' in x else None)
receipt_barcode_df['discountedItemPrice'] = receipt_barcode_df['rewardsReceiptItemList'].apply(lambda x: x.get('discountedItemPrice') if isinstance(x, dict) and 'discountedItemPrice' in x else None)
receipt_barcode_df = receipt_barcode_df[['_id','barcode','quantityPurchased','itemPrice','discountedItemPrice']]
receipt_barcode_df

,_id,barcode,quantityPurchased,itemPrice,discountedItemPrice
0,5ff1e1eb0a720f0523000575,4011,5.0,26.00,None
1,5ff1e1bb0a720f052300056b,4011,1.0,1,None
1,5ff1e1bb0a720f052300056b,028400642255,1.0,10.00,None
2,5ff1e1f10a720f052300057a,None,NaN,None,None
3,5ff1e1ee0a7214ada100056f,4011,4.0,28.00,None
...,...,...,...,...,...
1115,603d0b710a720fde1000042a,None,NaN,None,None
1116,603cf5290a720fde10000413,None,NaN,None,None
1117,603ce7100a7217c72c000405,B076FJ92M4,1.0,22.97,22.97
1117,603ce7100a7217c72c000405,B07BRRLSVC,1.0,11.99,11.99


*** 
### Experimenting with the common date format columns

In [66]:
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)

receipts_df['createDate_unixtime'] = receipts_df['createDate'].apply(lambda x: x['$date'])
receipts_df['createDate_DATE'] = receipts_df['createDate'].apply(lambda x: pd.to_datetime(x['$date'], unit='ms'))
receipts_df_sub = receipts_df[['createDate','createDate_unixtime', 'createDate_DATE']]
receipts_df_sub

,createDate,createDate_unixtime,createDate_DATE
0,{'$date': 1609687531000},1609687531000,2021-01-03 15:25:31.000
1,{'$date': 1609687483000},1609687483000,2021-01-03 15:24:43.000
2,{'$date': 1609687537000},1609687537000,2021-01-03 15:25:37.000
3,{'$date': 1609687534000},1609687534000,2021-01-03 15:25:34.000
4,{'$date': 1609687506000},1609687506000,2021-01-03 15:25:06.000
...,...,...,...
1114,{'$date': 1614594147000},1614594147000,2021-03-01 10:22:27.000
1115,{'$date': 1614613361873},1614613361873,2021-03-01 15:42:41.873
1116,{'$date': 1614607657664},1614607657664,2021-03-01 14:07:37.664
1117,{'$date': 1614604048000},1614604048000,2021-03-01 13:07:28.000


In [67]:
# Checking for NULLs if there is any data issues below
null_counts = receipts_df_sub.isnull().sum()
null_counts

createDate             0
createDate_unixtime    0
createDate_DATE        0
dtype: int64

***
### Testing the unique values of receipt status

In [73]:
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)

# Get unique values and their counts from the 'rewardsReceiptStatus' column
status_counts = receipts_df['rewardsReceiptStatus'].value_counts(dropna=False)
# Display the count of each status
status_counts


rewardsReceiptStatus
FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: count, dtype: int64

***
### Confirming Integers or Doubles for number columns

In [108]:
# List of columns to check
columns_to_check = ['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']

# Function to check for non-zero decimal parts
def check_decimal_values(df, columns):
    for column in columns:
        if column in df.columns:
            # Drop NaN values to avoid errors in modulo operation
            non_null_values = df[column].dropna()
            
            # Check for non-zero decimals by comparing the value with its floor
            non_zero_decimals = (non_null_values != non_null_values.astype(int))
            
            # Count non-zero decimals
            count_non_zero_decimals = non_zero_decimals.sum()

            # Print results
            print(f"Column '{column}' has {count_non_zero_decimals} values with non-zero decimals.")
        else:
            print(f"Column '{column}' does not exist in the DataFrame.")

check_decimal_values(receipts_df, columns_to_check)


Column 'bonusPointsEarned' has 0 values with non-zero decimals.
Column 'pointsEarned' has 65 values with non-zero decimals.
Column 'purchasedItemCount' has 0 values with non-zero decimals.
Column 'totalSpent' has 266 values with non-zero decimals.


In [109]:
# Manual visual check for pointsEarned column to see that it has decimals that are not '.0'

# Drop NaN values from 'pointsEarned' to ensure the comparison operates correctly
receipts_df = receipts_df.dropna(subset=['pointsEarned'])

# Filter the DataFrame to include only rows where 'pointsEarned' has non-zero decimals
pointsEarned_df = receipts_df[receipts_df['pointsEarned'] != receipts_df['pointsEarned'].astype(int)]

# Display the resulting subset DataFrame
pointsEarned_df


,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
20,5f9c74f90a7214ad07000038,300.0,"Receipt number 4 completed, bonus point schedu...",{'$date': 1604089081000},{'$date': 1604089081000},{'$date': 1609688395000},{'$date': 1609688395000},{'$date': 1609688395000},389.2,{'$date': 1604002681000},6.0,"[{'barcode': '075925306254', 'competitiveProdu...",FINISHED,14.00,5f9c74e3f1937815bd2c1d73
78,5fa5ad370a720f05ef000089,750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1604693303000},{'$date': 1604693303000},{'$date': 1609880020000},{'$date': 1609880020000},{'$date': 1609880020000},9449.8,{'$date': 1604606903000},11.0,"[{'barcode': '075925306254', 'competitiveProdu...",FINISHED,291.00,5fa5ad376a26f611e71ab5ef
88,5ff473bc0a720f05230005b8,NaN,NaN,{'$date': 1609855932000},{'$date': 1609855932000},{'$date': 1609855933000},{'$date': 1609855933000},{'$date': 1609855933000},91.2,{'$date': 1609769532000},4.0,"[{'barcode': '021000068760', 'description': 'F...",FINISHED,18.20,5ff47392c3d63511e2a47881
104,5ff5d1fa0a720f05230005dd,27.0,COMPLETE_PARTNER_RECEIPT,{'$date': 1609945594000},{'$date': 1609945594000},{'$date': 1609945594000},{'$date': 1609945594000},{'$date': 1609945594000},350.6,{'$date': 1609859194000},4.0,"[{'barcode': '043000013229', 'description': 'T...",FINISHED,4.66,5ff5d15aeb7c7d12096d91a2
108,5ff618e30a7214ada10005fa,750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1609963747000},{'$date': 1609963747000},{'$date': 1609963782000},{'$date': 1609963785000},{'$date': 1609963782000},793.1,{'$date': 1609891200000},16.0,[{'description': 'EMIL' S SAUSAGE MUSHROOM PIZ...,FINISHED,52.12,5ff616a68f142f11dd189163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,6014853a0a7214ad50000106,NaN,NaN,{'$date': 1611957562000},{'$date': 1611957562000},{'$date': 1611957563000},{'$date': 1612201634000},{'$date': 1611957563000},209.5,{'$date': 1611878400000},1.0,"[{'barcode': '036000320893', 'description': 'H...",FINISHED,20.95,60147d2ac8b50e11d8453f53
725,601aaf710a720f05f40001e6,27.0,COMPLETE_PARTNER_RECEIPT,{'$date': 1612361585000},{'$date': 1612361585000},{'$date': 1612361586000},{'$date': 1612361586000},{'$date': 1612361586000},50.6,{'$date': 1612275185000},4.0,"[{'barcode': '029000019416', 'description': 'P...",FINISHED,4.66,601aaebde409201029849b3c
793,601ead690a720f053c000092,27.0,COMPLETE_PARTNER_RECEIPT,{'$date': 1612623209000},{'$date': 1612623209000},{'$date': 1612623210000},{'$date': 1612623210000},{'$date': 1612623210000},50.6,{'$date': 1612536809000},4.0,"[{'barcode': '043000058404', 'description': 'G...",FINISHED,4.66,601ead00b57b8a12187fce78
803,601ead5b0a720f053c00008f,27.0,COMPLETE_PARTNER_RECEIPT,{'$date': 1612623195000},{'$date': 1612623195000},{'$date': 1612623196000},{'$date': 1612623196000},{'$date': 1612623196000},50.6,{'$date': 1612536795000},4.0,"[{'barcode': '087684004449', 'description': 'C...",FINISHED,4.66,601ead00b57b8a12187fce78


***
### Checking Unique status of receipt _ids

In [110]:
receipts_df = pd.read_json(r"JSON Data\receipts.json.gz", compression='gzip', lines=True)
# Step 1: Extract '$oid' from the _id column
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)

# Step 2: Extract '$date' from various date columns
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for column in date_columns:
    receipts_df[column] = receipts_df[column].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else None)

# Display the cleaned DataFrame
receipts_df

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,NaN,1614594148000,NaN,25.0,1.597622e+12,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,1614613361873,1614613361873,NaN,1614613361873,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,1614607657664,1614607657664,NaN,1614607657664,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,NaN,1614604049000,NaN,25.0,1.597622e+12,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [111]:

# Step 1: Group by '_id' and count the number of rows for each '_id'
id_counts_df = receipts_df.groupby('_id').size().reset_index(name='count')

# Step 2: Optionally, sort the DataFrame by 'count' if you want to see the most frequent '_id' first
id_counts_df_sorted = id_counts_df.sort_values(by='count', ascending=False).reset_index(drop=True)

# Display the resulting DataFrame
id_counts_df_sorted


,_id,count
0,5f9c74f70a7214ad07000037,1
1,601c27870a720f05f400029e,1
2,601c540f0a720f05f40002be,1
3,601c40230a720f05f40002b5,1
4,601c2d630a7214ad280002a7,1
...,...,...
1114,60088a460a720f05fa0000d5,1
1115,60088a490a720f05fa0000d6,1
1116,60088a4a0a7214ad890000c5,1
1117,60088a4d0a7214ad890000c7,1


***
## Users

In [12]:
users_df

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


***
## Brands

In [13]:
brands_df

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


In [45]:
# Standard check for NULLS in the dataframe to see what columns are the most consistent:
brands_df.isnull().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

***
### CPG Analysis

The `cpg` column is foreign to me, usually I understand CPG as a specific type of products with these traits:

 - Consumers use up and need to replace frequently
 - Are typically consumed regularly and bought at regular intervals
 - Typically low cost

So I want to see if every brand provided has a CPG ID associated with it, because not all brands sell CPG products

In [57]:
# Assuming brands_df is already defined and loaded

# Step 1: Extract $oid from the cpg column
brands_df['cpg_oid'] = brands_df['cpg'].apply(lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x and '$oid' in x['$id'] else None)

# Step 2: Count unique $oid values
unique_oid_count = brands_df['cpg_oid'].nunique()

# Step 3: Count nulls in the extracted $oid values
null_count = brands_df['cpg_oid'].isnull().sum()

# Print the results
print("Number of unique $oid values:", unique_oid_count)
print("Number of nulls in $oid values:", null_count)

Number of unique $oid values: 196
Number of nulls in $oid values: 0


Since the brands_df has no NULL `$oid` within the cpg column, that might mean that eithher:
1. Every brand provided is a brand that typically sells CPG type products
2. The use of the acronym "cpg" is different than my typical definition of it

I will check to see all of the unique categories provided in the sample to inform me which of these assumptions might be more correct

In [43]:
# Getting all of the unique brands to see if there is 
unique_categories = brands_df['category'].unique()
print(unique_categories)

['Baking' 'Beverages' 'Candy & Sweets' 'Condiments & Sauces'
 'Canned Goods & Soups' nan 'Magazines' 'Breakfast & Cereal'
 'Beer Wine Spirits' 'Health & Wellness' 'Beauty' 'Baby' 'Frozen'
 'Grocery' 'Snacks' 'Household' 'Personal Care' 'Dairy'
 'Cleaning & Home Improvement' 'Deli' 'Beauty & Personal Care'
 'Bread & Bakery' 'Outdoor' 'Dairy & Refrigerated']


Most of these categories seem to fit within the traditional 'CPG' space, with the potential exception fo 'Outdoor' but otherwise I am leaning towards all of the brands provided in the sample are within the CPG space.

I will do one more check to see if there is a 1:1 link between the category and the CPG code in the brands_df dataframe:

In [50]:
# Step 1: Extract $oid from the cpg column
brands_df['cpg_oid'] = brands_df['cpg'].apply(lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x and '$oid' in x['$id'] else None)

# Step 2: Group by category and aggregate unique $oid values
category_test_df = brands_df.groupby('category')['cpg_oid'].agg(lambda x: set(x.dropna())).reset_index()

# Renaming the column after aggregation to 'distinct_oids' for clarity
category_test_df.rename(columns={'cpg_oid': 'distinct_oids'}, inplace=True)

# Step 3: Adding a size of unique cpg_oids
category_test_df['count_distinct_oids'] = category_test_df['distinct_oids'].apply(len)

# Display the resulting DataFrame
category_test_df

,category,distinct_oids,count_distinct_oids
0,Baby,"{5d9b4f591dda2c6225a284aa, 55b62995e4b0d8e685c...",9
1,Baking,"{5ff74500be37ce1e961f3276, 5ffe3381be37ce5e017...",149
2,Beauty,"{5332f5f6e4b03c9a25efd0b4, 5e28b81cee7f2d697e8...",2
3,Beauty & Personal Care,{53e10d6368abd3c7065097cc},1
4,Beer Wine Spirits,"{53e10d6368abd3c7065097cc, 5fb6d381be37ce522e1...",3
5,Beverages,"{55b62995e4b0d8e685c14213, 559c2234e4b06aca36a...",7
6,Bread & Bakery,{5fb6b608be37ce522e165cb9},1
7,Breakfast & Cereal,"{559c2234e4b06aca36af13c6, 5332f5f3e4b03c9a25e...",5
8,Candy & Sweets,{5332fa12e4b03c9a25efd1e7},1
9,Canned Goods & Soups,"{5332f5f6e4b03c9a25efd0b4, 5a734034e4b0d58f376...",2


There is definitely many CPG ID's per category - I will try the same with brands to be extra sure

In [51]:
brands_df = pd.read_json(r"JSON Data\brands.json.gz", compression='gzip', lines=True)

# Step 1: Extract $oid from the cpg column
brands_df['cpg_oid'] = brands_df['cpg'].apply(lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x and '$oid' in x['$id'] else None)

# Step 2: Group by name and aggregate unique $oid values
name_test_df = brands_df.groupby('name')['cpg_oid'].agg(lambda x: set(x.dropna())).reset_index()

# Renaming the column after aggregation to 'distinct_oids' for clarity
name_test_df.rename(columns={'cpg_oid': 'distinct_oids'}, inplace=True)

# Step 3: Adding a size of unique cpg_oids
name_test_df['count_distinct_oids'] = name_test_df['distinct_oids'].apply(len)

name_test_df


,name,distinct_oids,count_distinct_oids
0,.,{581ce8d5e4b058c71f74c442},1
1,1915 Bolthouse Farms,{5a734034e4b0d58f376be874},1
2,1_KRAFT Hockeyville,{53e10d6368abd3c7065097cc},1
3,7 up,{55b62995e4b0d8e685c14213},1
4,A&W Rootbeer,{55b62995e4b0d8e685c14213},1
...,...,...,...
1151,test brand @1612366146091,{601ac142be37ce2ead437559},1
1152,test brand @1612366146133,{601ac142be37ce2ead437559},1
1153,test brand @1612366146176,{601ac142be37ce2ead437559},1
1154,test brand @1612366146827,{5332fa12e4b03c9a25efd1e7},1


In the preview, this looks promising since all of them have 1's, so I will aggregate to see how many brand names there are that have only 1 by aggregating it up:

In [52]:
oid_counts_df = name_test_df.groupby('count_distinct_oids').size().reset_index(name='number_of_brands')

# Renaming columns for clarity
oid_counts_df.columns = ['count_distinct_oids', 'number_of_brands']

oid_counts_df

,count_distinct_oids,number_of_brands
0,1,1148
1,2,8


There are 8 brands that have multiple oid's within them, which isn't a lot, but this might prove my initial hypothesis wrong that the cpg oid is not a code for a brand name

In [53]:
# Subsetting the dataframe to get the number of brand names that had more than one oid
# I know this may be incorrect to use the name instead of the brandCode, but brandCode seemed to have more NULL's
# So for my local analysis, I thought it would be better to use the name instead as a quick check
name_test_df_sub = name_test_df[name_test_df['count_distinct_oids'] > 1]
name_test_df_sub

,name,distinct_oids,count_distinct_oids
129,Caleb's Kola,"{5332f5fbe4b03c9a25efd0ba, 53e10d6368abd3c7065...",2
215,Diabetic Living Magazine,"{53e10d6368abd3c7065097cc, 5d5d4fd16d5f3b23d1b...",2
223,Dippin Dots® Cereal,"{5332f5f3e4b03c9a25efd0ae, 53e10d6368abd3c7065...",2
313,Health Magazine,"{5d66b9dcee7f2d201c7281cd, 5d5d4fd16d5f3b23d1b...",2
333,Huggies,"{550b2565e4b001d5e9e4146f, 5459429be4b0bfcb1e8...",2
335,I CAN'T BELIEVE IT'S NOT BUTTER!,"{53e10d6368abd3c7065097cc, 5e5ea9cc9c50da1291a...",2
627,Sierra Mist,"{5332f5fbe4b03c9a25efd0ba, 53e10d6368abd3c7065...",2
699,V8 Hydrate,"{5a734034e4b0d58f376be874, 53e10d6368abd3c7065...",2


After a final visual check, I think the cpg column must be some subcategory of ID per brand, or some brands can be in multiple CPG categories. I will just have CPG's separated to their own table to be safe

***
### Barcode to cpg_id_oid analysis
I later realize that the real relationship is more likley to be between the product (barcode) and the cpg category, I am going to do a quick analysis on that relationship to confirm that hypothesis

In [133]:
brands_df = pd.read_json(r"JSON Data\brands.json.gz", compression='gzip', lines=True)
# Assuming brands_df is already defined and includes the 'barcode' and 'cpg' columns

# Step 1: Extract '$oid' from the nested dictionary in the 'cpg' column
brands_df['cpg_id_oid'] = brands_df['cpg'].apply(
    lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x and '$oid' in x['$id'] else None
)

# Grouping by 'barcode', aggregate unique 'cpg_id_oid' into sets, and count them
brands_cpg = brands_df.groupby('barcode').agg(
    cpgs=('cpg_id_oid', lambda ids: set(ids.dropna())),  # Create sets of unique cpg_id_oid
    count_of_distinct_cpg_id_oid=('cpg_id_oid', lambda ids: len(set(ids.dropna())))  # Count unique cpg_id_oid
).reset_index()

 # Sorting the data for visual check
brands_cpg = brands_cpg.sort_values(by='count_of_distinct_cpg_id_oid', ascending=False)

brands_cpg = brands_cpg[brands_cpg['count_of_distinct_cpg_id_oid'] > 1].reset_index(drop=True)
brands_cpg


,barcode,cpgs,count_of_distinct_cpg_id_oid
0,511111504139,"{5a734034e4b0d58f376be874, 55b62995e4b0d8e685c...",2
1,511111305125,"{55b62995e4b0d8e685c14213, 5d5d4fd16d5f3b23d1b...",2
2,511111704140,"{5a734034e4b0d58f376be874, 55b62995e4b0d8e685c...",2
3,511111605058,"{5d5d4fd16d5f3b23d1bc7905, 5c45f8b087ff3552f95...",2
4,511111004790,"{55b62995e4b0d8e685c14213, 559c2234e4b06aca36a...",2
5,511111504788,"{59ba6f1ce4b092b29c167346, 559c2234e4b06aca36a...",2
6,511111204923,"{5332f5fbe4b03c9a25efd0ba, 5c45f8b087ff3552f95...",2


This also just has 7 barcodes that share 2 cpg's - so it's has a similar level of granularity to the brands and cpg's. 

Since barcode is a more granular level than brands generally - I think it feels safer to make a separate table relating `barcode` to the `cpg`

***
Checking for the relationship between `barcode` and `_id` within the brands file

In [62]:
brands_df = pd.read_json(r"JSON Data\brands.json.gz", compression='gzip', lines=True)

# Step 1: Extract $oid from the _id column
brands_df['oid'] = brands_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)

# Step 2: Group by barcode and aggregate unique $oid values
barcode_brand_count = brands_df.groupby('barcode').agg({
    'oid': [('number_of_unique_brands', 'nunique'), ('unique_brands', lambda x: list(set(x.dropna())))]
}).reset_index()

# Step 3: Flatten the column headers resulting from the aggregation
barcode_brand_count.columns = ['barcode', 'number_of_unique_brands', 'unique_brands']

# Display the resulting DataFrame
barcode_brand_count


,barcode,number_of_unique_brands,unique_brands
0,511111000167,1,[5a5d2727e4b0db471c2d042e]
1,511111000235,1,[5a5d247ae4b06ba572cf24a2]
2,511111000396,1,[5a5d1bffe4b0db471c2d0423]
3,511111000440,1,[5a5d17ede4b06ba572cf2498]
4,511111000518,1,[5a4d23dae4b0bcb2c74ea77e]
...,...,...,...
1155,511111919476,1,[5ffe3686be37ce5e01754c61]
1156,511111919520,1,[5ffe3a86be37ce7aab2d4fd8]
1157,511111919681,1,[6000b8c3be37ce321363c6c1]
1158,511111919759,1,[60019b82be37ce4f17e558d4]


In [63]:
# Manual Visual check for brands that have more than 1 brand_id in them
barcode_brand_count_sub = barcode_brand_count[barcode_brand_count['number_of_unique_brands'] > 1]
barcode_brand_count_sub

,barcode,number_of_unique_brands,unique_brands
46,511111004790,2,"[5c409ab4cd244a3539b84162, 5cdacd63166eb33eb7c..."
282,511111204923,2,"[5d6027f46d5f3b23d1bc7906, 5c45f91b87ff3552f95..."
400,511111305125,2,"[5c4699f387ff3577e203ea29, 5d642d65a3a01851499..."
622,511111504139,2,"[5a7e0604e4b0aedb3b84afd3, 5a8c33f3e4b07f0a2da..."
628,511111504788,2,"[5c408e8bcd244a1fdb47aee7, 5ccb2ece166eb31bbba..."
745,511111605058,2,"[5d6415d5a3a018514994f429, 5c4637ba87ff35681e8..."
852,511111704140,2,"[5a7e0665e4b0aedb3b84afd4, 5a8c344ae4b0ccf165f..."


There are 7 barcodes which have 2 brands associated with them, which could be a data issue.

***

Checkign the _id column

In [56]:
# Using apply() to extract $oid values to a new series
oids_series = brands_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else None)

# Count unique $oid values using nunique()
number_of_unique_oids = oids_series.nunique()

null_oids = oids_series.isnull().sum()

# Print the number of unique $oid values
print("Number of unique $oid values:", number_of_unique_oids)

print(f"Number of null oid values: {null_oids}")

Number of unique $oid values: 1167
Number of null oid values: 0


In [5]:
# Checking the unique values of topBrand
print(brands_df['topBrand'].unique())

[ 0. nan  1.]


Since the topBrand column does have NaN values in it, it is good to keep this value as an integer inside of the database, so we can have 0, 1 and NULL for unlcear data for if it is a top brand

I also wanted to check the kinds of keys inside of the cpg column to make sure I wasn't missing anything

In [6]:
# Getting all distinct keys inside of the cpg column
distinct_keys = set()
for item in brands_df['cpg']:
    if isinstance(item, dict):
        distinct_keys.update(item.keys())

# Print the distinct keys found
print(distinct_keys)


{'$id', '$ref'}


Checking what kind of data can be contained inside of the ID key

In [58]:
# Using apply() to extract $ref values
cpg_ref = brands_df['cpg'].apply(lambda x: x.get('$ref') if isinstance(x, dict) and '$ref' in x else None)

cpg_set = set()

for item in cpg_ref:
    cpg_set.add(item)

print(cpg_set)

{'Cogs', 'Cpgs'}


There is only 2 kinds of refrences, so this is something that would need to be clarified for a further analysis, and will be ignored for now, but it must be some kind of source type for where the ID came from

In [38]:
# Getting the number of unique values from cpg
# Using apply() to extract $oid values and collect them into a set
unique_oids = set(brands_df['cpg'].apply(lambda x: x['$id']['$oid'] if isinstance(x, dict) and '$id' in x and isinstance(x['$id'], dict) and '$oid' in x['$id'] else None).dropna())

# Print the set of unique $oid values
print(len(unique_oids))

196


In [31]:
# Getting the number of unique categoryCode's
# Count unique values in the 'categoryCode' column
number_of_unique_category_codes = brands_df['brandCode'].nunique()

# Print the result
print("Number of unique category codes:", number_of_unique_category_codes)


Number of unique category codes: 897


***
# Data Quality Check 

## Brands
Within the `brands_df` (the dataframe sourced from `brands.json.gz`) there is an inconsistent map in the `cpg` column, which has 3 potential types:
 - `{'$ref':'Cogs'}`
 - `{'$ref':'Cpgs'}`
 - Dictionary does not contain any `'$ref'` key at all

It would be very helpful to clear up what these sources are so it could be properly labeled. I think this would be worthwhile to make a new column with this data called `cpg_source` which could contain either the 3 values:
 - `'Cogs'`
 - `'Cpgs'`
 - `'Missing'` or `Null` for the empty one

***

Additionally, there seems to be more code's that are missing more frequently than the typically reader friendly columns. For example, between `name` and `brandCode`, the brandCode seems to have more NULLs than name. This is problematic becuase the brandCode should be the source for a foreign key in other tables, and would cause problems if there is missing values.

We see this same issue with `category` and `categoryCode` where the code (likely the better suited foreign key) is NULL more frequently than the `category` user-friendly column. Both of these are large issues with creating new data, because without a proper code, there is no reliable way to join it to other tables, since using the user fieldnly columns like `category` and `name` could not be consistent.

***
I also found that within the `brands.json.gz` file, there appears to be 7 barcode's that contain 2 brands linked to them. I would assume this is a data quality issue since the items should not belong to multiple brands

## Receipts
Inside of the `rewardsReceiptItemList`, there is a good number of NAN values for the barcode values
 - Sometimes both the `description` and `barcode` is NULL
 - Sometiems the `description` is present but the `barcode` is NULL